In [109]:
import glob
import time
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

In [229]:
files = glob.glob('data/Unidirectional/Combined/*.csv')
files.extend(glob.glob('data/Bidirectional/Combined/*.csv'))

In [228]:
bidirectional = glob.glob('data/Bidirectional/*.csv')

subjects = set()
for file in bidirectional:
    subjects.add(file[file.rfind('/')+1:][:3])

for subject in subjects:
    data = glob.glob(f'data/Bidirectional/*{subject}*_Adapt.csv')
    data = [pd.read_csv(d, index_col=0) for d in data]
    pd.concat(data).to_csv(f'data/Bidirectional/Combined/{subject}_Adapt.csv')

In [235]:
def f(x, m, s):
    return norm.cdf(x, m, s)

output = pd.DataFrame(columns=['Name', 'Exp Type', 'Mu', 'Sigma', 'Direction'])

for file in files:
    
    data = pd.read_csv(file, index_col=0)

    count_data = data.groupby('Test Speed')['Response'].apply(lambda x: x[x.str.contains('right')].count())
    total_counts = data.groupby('Test Speed').count()['Response']
    speeds = count_data.index.tolist()
    t = np.linspace(min(speeds), max(speeds), 1000)

    right_counts = count_data.tolist()
    right_props = [r / c for r, c in zip(right_counts, total_counts)]

    mu, sigma = curve_fit(f, speeds, right_props, p0=[0, 2], method='dogbox')[0]
    
    name = file[file.rfind('/')+1:][:3].upper()
    t_type = 'Unidirectional' if 'Unidirectional' in file else 'Bidirectional'
    if 'noAdapt' in file:
        t_type = 'No Adapt'
    elif 'practice' in file:
        t_type = 'Practice'
    else:
        t_type += ' Adapt'
    
    if 'noAdapt' in file or 'practice' in file:
        direction = ''
    elif 'Bidirectional' in file:
        direction = 0
    else:
        direction = -1 if min(speeds) < -40 else 1
        #print(f'{name}: {min(speeds)}, {max(speeds)}\t{direction}')
    
    output.loc[len(output)] = [name, t_type, mu, sigma, direction]
    
    if 'practice' in file:
        continue
    
    #plt.plot(speeds, right_props, alpha=.5, label='Subject')
    plt.figure(figsize=(8,6))
    plt.scatter(speeds, right_props, color='orange', label='Subject Data')
    plt.plot(t, norm.cdf(t, mu, sigma), alpha=.5, color='blue', label='Guassian Fit')
    plt.axhline(y=0.5, color='r', linewidth=0.5, linestyle='dashed')
    plt.axvline(x=0, color='r', linewidth=0.5, linestyle='dashed')
    plt.ylim(-0.05, 1.05)
    plt.xlabel('Test Stimulus Speed (deg/s)')
    plt.ylabel('Proportion of Responding Clockwise')
    plt.title(f'{name} ({t_type})')
    plt.legend(loc='lower right', framealpha=0.5)
    
    filename = file[file.rfind('/'):-4]
    plt.savefig(f'plots/{filename}.pdf')
    plt.close()

output = output.sort_values(['Name', 'Exp Type']).reset_index(drop=True)

NXN: -10.0, 70.0	1
KSM: -70.0, 10.0	-1
TEE: -70.0, 10.0	-1
SXK: -40.0, 70.0	1
MJL: -10.0, 70.0	1
JDG: -70.0, 10.0	-1
NST: -70.0, 10.0	-1
TXD: -10.0, 70.0	1
KXT: -10.0, 70.0	1
MXW: -70.0, 10.0	-1
ANB: -10.0, 70.0	1
NXS: -70.0, 10.0	-1


In [236]:
output.to_csv('GaussFitData.csv')